In [1]:
# imports
import pandas as pd
import os
import tiktoken
import openai
from dotenv import load_dotenv

from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
qa_df = pd.read_csv("data/crland.csv")

In [4]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [5]:
qa_df = qa_df.dropna()

In [6]:
qa_df["combined"] = ("Prompt: " + qa_df["prompt"] + "\n\nCompletion: " + qa_df["completion"] +"\n\n")

In [7]:
encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
qa_df["n_tokens"] = qa_df.combined.apply(lambda x: len(encoding.encode(x)))


In [8]:
qa_df

,prompt,completion,combined,n_tokens
0,华润城整体规划单位及其知名作品,RTKL国际有限公司（RTKL International）。是世界上最大的建筑规划设计公司...,Prompt: 华润城整体规划单位及其知名作品\n\nCompletion: RTKL国际有...,233
1,公建区规划及设计单位,公建区整体规划及展示中心设计单位：福斯特 (Foster)。参与的设计公司：RTKL、Fos...,Prompt: 公建区规划及设计单位\n\nCompletion: 公建区整体规划及展示中心...,276
2,建筑施工单位与建筑设计单位,建筑施工单位：中建三局。建筑设计单位：CCDI。\n创立于1994年，是在世界城市建设和开发...,Prompt: 建筑施工单位与建筑设计单位\n\nCompletion: 建筑施工单位：中建...,474
3,万象天地园林设计单位及其曾设计过的知名作品,James Corner Field Operations（JCFO）事务所\n简介：该事务...,Prompt: 万象天地园林设计单位及其曾设计过的知名作品\n\nCompletion: J...,186
4,精装修设计单位,西塔：美国WILSON公司、东塔Kokai。简介：美国WILSON室内设计公司成立于1971...,Prompt: 精装修设计单位\n\nCompletion: 西塔：美国WILSON公司、东...,426
...,...,...,...,...
91,停车收费标准？临时停放的车辆如何收费？,待定,Prompt: 停车收费标准？临时停放的车辆如何收费？\n\nCompletion: 待定\n\n,32
92,物业管理公司的有偿和无偿服务项目？每日服务时间？,待定,Prompt: 物业管理公司的有偿和无偿服务项目？每日服务时间？\n\nCompletion...,29
93,物业服务内容是什么？,待定,Prompt: 物业服务内容是什么？\n\nCompletion: 待定\n\n,18
94,入伙时须交哪些费用及金额计算？,物业管理收费应按月收取，经双方协商约定可以预收，但预收期限不得超过_____个月。\n本体维...,Prompt: 入伙时须交哪些费用及金额计算？\n\nCompletion: 物业管理收费应...,102


In [9]:
# This may take a few minutes
qa_df["embedding"] = qa_df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))

In [10]:
qa_df.to_csv("data/crland_embeddings_v2.csv",encoding='utf-8')

In [11]:
qa_df.head()

,prompt,completion,combined,n_tokens,embedding
0,华润城整体规划单位及其知名作品,RTKL国际有限公司（RTKL International）。是世界上最大的建筑规划设计公司...,Prompt: 华润城整体规划单位及其知名作品\n\nCompletion: RTKL国际有...,233,"[0.004536231514066458, 0.0005073416978120804, ..."
1,公建区规划及设计单位,公建区整体规划及展示中心设计单位：福斯特 (Foster)。参与的设计公司：RTKL、Fos...,Prompt: 公建区规划及设计单位\n\nCompletion: 公建区整体规划及展示中心...,276,"[0.003463658271357417, -0.00806100107729435, -..."
2,建筑施工单位与建筑设计单位,建筑施工单位：中建三局。建筑设计单位：CCDI。\n创立于1994年，是在世界城市建设和开发...,Prompt: 建筑施工单位与建筑设计单位\n\nCompletion: 建筑施工单位：中建...,474,"[-0.006669289898127317, 0.005116167478263378, ..."
3,万象天地园林设计单位及其曾设计过的知名作品,James Corner Field Operations（JCFO）事务所\n简介：该事务...,Prompt: 万象天地园林设计单位及其曾设计过的知名作品\n\nCompletion: J...,186,"[0.0022138275671750307, -0.006464446894824505,..."
4,精装修设计单位,西塔：美国WILSON公司、东塔Kokai。简介：美国WILSON室内设计公司成立于1971...,Prompt: 精装修设计单位\n\nCompletion: 西塔：美国WILSON公司、东...,426,"[0.005364165175706148, -0.010392632335424423, ..."


In [12]:
prompt = "用自然语言介绍一下大沙河创新走廊"

In [13]:
response = openai.Embedding.create(
    input = prompt,
    model = "text-embedding-ada-002"
)
promptembedding = response['data'][0]['embedding']

In [14]:
qa_df["similarity"] = qa_df.embedding.apply(lambda x: cosine_similarity(x, promptembedding))

In [15]:
qa_df_similarity = qa_df.sort_values(by="similarity", ascending=False)

In [16]:
qa_df_similarity.to_csv("data/crland_embeddings_similarity_v2.csv",encoding='utf-8')

In [17]:
qa_df_similarity = qa_df_similarity.reset_index(drop=True)

In [18]:
for i in range(5):
    print(qa_df_similarity.at[i, "prompt"])

大沙河创新走廊
大沙河生态廊道（规划中）
沙河西路
大沙河公园
万象府的建筑景观情况


In [19]:
messages = []
for i in range(5):
    message = {"role": "assistant", "content": qa_df_similarity.at[i, "combined"]}
    messages.append(message)

In [20]:
message = {"role": "user", "content": "根据告诉你的信息，回答：" + prompt}
messages.append(message)

In [21]:
messages

[{'role': 'assistant',
  'content': 'Prompt: 大沙河创新走廊\n\nCompletion: 华润城项目地处大沙河创新走廊的下游，属于南山市政府重点规划区域。\n大沙河创新走廊是深圳市“十二五”规划的重大项目，目前已进入第二期开发阶段，大沙河创新走廊的规划范围覆盖大沙河全流域，包括西丽、桃源街道全部，粤海街道的滨海大道以北部分，沙河街道的深南大道以北部分，以及南头街道的马家龙片区。共计101平方公里。从大沙河源头福龙大道起，到滨海大道深圳湾入海口止，通过大沙河流域内旧城、旧村、旧工业区改造，在上游以大学城为核心建设知识创新高地，中游以留仙洞为核心建设产业创新高地，下游以深圳湾高新区为核心建设技术创新高地，连接成为深圳市建设国家核心技术自主创新先锋城市的核心区。\n大沙河创新走廊未来将建成创新型企业总部、研发基地和创新型人才宜聚地，同时未来的大沙河流域将成为南山次级行政中心和商业文化中心。\n\n'},
 {'role': 'assistant',
  'content': 'Prompt: 大沙河生态廊道（规划中）\n\nCompletion: 大沙河是南山区惟一的一条河流，也是南山的“母亲河”，起始于长岭皮水库，穿越城区汇入深圳湾，全长13.5公里，流域面积90.69平方公里。\n规划创造亲近自然的城市滨水空间，同时成为大沙河创新走廊的“一条文化中心线”，有关部门正对大沙河全程疏浚、修闸，并在东岸沿河建造“大沙河文化长廊”，使之成为深圳的城市名片，成为联结塘朗山郊野公园生态系统与深圳湾生态系统的生态廊道。\n\n'},
 {'role': 'assistant',
  'content': 'Prompt: 沙河西路\n\nCompletion: 沙河西路是南山区南北向最长的路。是南山大道的2倍、南海大道的1.5倍长，长约12公里南至滨海大道，无缝连接深港跨海大桥，北至白芒检查站，贯穿了南山区西丽街道。\n交汇道路：北环大道、深南路、白石路、滨海大道。\n沿途商圈：后海中心区。\n\n'},
 {'role': 'assistant',
  'content': 'Prompt: 大沙河公园\n\nCompletion: 大沙河公园是综合市政公园，被深圳市城市管理局评为“四星级公园”。公园位于南山区中部，北邻广（州）深（圳

In [22]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages
)

In [23]:
response = completion.choices[0].message.content

In [24]:
response

'大沙河创新走廊是深圳市的一个重大规划项目，涵盖了西丽、桃源、沙河和南头等地区，总面积达到101平方公里。它的规划目标是通过旧城、旧村和旧工业区的改造来推进深圳市的跨越式发展，打造出一个南山区的核心区。该项目分为三个部分，上游以大学城为核心建设知识创新高地，中游以留仙洞为核心建设产业创新高地，下游以深圳湾高新区为核心建设技术创新高地，连接成为深圳市建设国家核心技术自主创新先锋城市的核心区。该项目的目标是建立一个创新型企业总部、研发基地和创新型人才宜聚地，同时将大沙河流域打造成南山的商业文化中心和城市名片。'

In [46]:
prompt =f""" 
```\n
苏联模式是一种集中式、计划经济体制的经济模式，即指苏联在20世纪实行的社会主义经济体制。苏联模式的特点包括国有垄断、计划经济、等价交换、低效率、不合理的资源配置等。苏联模式的主要原则是政府控制国有企业和资源，并通过中央计划来指导生产和分配。该模式在苏联及其卫星国家使用了几十年时间，在经济上取得了一定的成功，但也面临了很多问题和挑战，最终在1991年苏联解体后终结。

苏联模式是指苏联在20世纪的一套政治、经济、文化的制度和模式，其特点包括计划经济、集权主义、政治权力高度集中和国有化等。这种模式在苏联领导人斯大林时期得到了广泛的推广和实施，但随着时间的推移，这种模式逐渐暴露出许多问题。

首先，计划经济效率低下。计划经济需要依靠一套庞大的机构和计算模型进行计划安排，但由于市场与价格机制的缺失，无法及时反映生产和消费的需求变化，导致生产效率低下、产出品质差，大量浪费社会资源。

其次，集权主义导致决策不民主、无法有效响应社会变化。在苏联，政治权力高度集中，政府掌握了所有的经济、政治和文化资源，导致政府缺少对市场和民间经济的了解和把握，对自由市场竞争缺乏适应性，不能满足人民的需求和要求，使得国家和民间经济难以协同发展。

此外，国有化也引起了一系列问题。在苏联，实行国有化，使得国家几乎垄断了所有工业、农业和服务业，使得政府需要大量的资源来维持，导致财政状况严峻，而由于国有化所造成的资源浪费和不必要的开支，也使得国家经济长期处于亏损状态。

最后，苏联模式对人的自由与权利的限制过于严格。集权主义的体制使得个人自由被剥夺，特别是言论、新闻和出版自由受到了很大限制。这样的体制导致知识和信息的匮乏，影响了人们的思想和素质成长。这些问题最终导致苏联在1989年解体。

苏联模式是指共产主义国家中的集权、计划经济、一党专政、个人权利受限等特征。为了防止苏联模式重现，以下是一些建议：

1. 推动政治制度的改革，建立法治国家，保护人民的自由和权利，实现政治多元化。

2. 建立市场经济，并实行合理有效的市场调节和竞争机制，以保持经济发展的稳定和可持续性。

3. 加强教育，提高人民的素质和觉悟，使其能够理性思考问题并参与民主决策。

4. 加强跨国合作和交流，促进国际关系的和平化和多元化。

5. 加强舆论监督和反腐斗争，保护国家利益和人民权利。

6. 高度重视科技革命和现代化建设，提高国家的竞争力。

总之，防止苏联模式重现需要在政治、经济、文化和社会等方面全面发展，不断推动社会进步和全球化发展。

苏联模式的遗留问题比较复杂，需要进行系统性的改革和重建。以下是一些可能的解决方案：

1.逐步开放市场经济：采取“市场化改革”措施，逐步开放市场，使资源的配置更加有效率。

2.改革政府管理机制：建立透明、公正、高效的政府管理机制，并加强政府与民间社会组织的合作与沟通。

3.重视教育文化：加强基础教育、职业培训和科技创新，赋予人们更多的知识和技能，提高其竞争力。

4.打击腐败：采取严厉的法律手段打击腐败现象。

5.公正司法：建立公正、高效的司法系统，维护人民的合法权益。

中国制定的社会主义建设方针相比苏联模式有以下几点优点：

1.强调农业优先发展：在苏联模式中，农业被忽视，工业被过分强调，导致农业生产水平低下，食品、物资短缺，农村贫困问题长期存在。而中国在制定社会主义建设方针时，明确强调“农业是国民经济的基础”，提出“以农业为基础、以工业为主导”的方针，促进了农业和工业的协调发展，保证了经济的稳定持续发展。

2.注重群众积极性：苏联模式中，社会主义建设主要依赖计划经济体制，缺乏市场机制，政府过于集中，主导决策，造成了效率低下、静止不前的问题。中国在制定社会主义建设方针时，重视调动群众积极性，发挥人民群众的主动性和创造性，注重市场机制的作用，鼓励个人和企业的自主创业和发展。

3.强调珠联璧合的地方发展：苏联的社会主义发展主要是集中在莫斯科和圣彼得堡等大城市，各地发展不平衡。中国在制定社会主义建设方针时，提出了“珠联璧合”的地方发展战略，即发挥各地区的优势，形成互补性的地方性经济，促进各地区经济均衡发展。

4.灵活运用马克思主义原则：中国在制定社会主义建设方针时，在继承马克思主义理论的基础上，根据我国实际情况进行创新，使建设方针更具针对性和实效性，更加符合国情。同时，为了适应时代的要求，中国不断调整和改进社会主义建设方针，实现了社会主义建设的稳步发展。
```
"""


In [47]:
messages = []
message = {"role": "user", "content": "以一个国际政治专家的角度，把下面的内组织成一篇文章，1000字。内容用三个`包括" + prompt}
messages.append(message)

In [36]:
messages[0]

{'role': 'user',
 'content': '以一个国际政治专家的角度，把下面的内组织成一篇论文。 以一个国际政治专家的角度，把下面的内组织成一篇论文。\n苏联模式是一种集中式、计划经济体制的经济模式，即指苏联在20世纪实行的社会主义经济体制。苏联模式的特点包括国有垄断、计划经济、等价交换、低效率、不合理的资源配置等。苏联模式的主要原则是政府控制国有企业和资源，并通过中央计划来指导生产和分配。该模式在苏联及其卫星国家使用了几十年时间，在经济上取得了一定的成功，但也面临了很多问题和挑战，最终在1991年苏联解体后终结。\n\n苏联模式是指苏联在20世纪的一套政治、经济、文化的制度和模式，其特点包括计划经济、集权主义、政治权力高度集中和国有化等。这种模式在苏联领导人斯大林时期得到了广泛的推广和实施，但随着时间的推移，这种模式逐渐暴露出许多问题。\n\n首先，计划经济效率低下。计划经济需要依靠一套庞大的机构和计算模型进行计划安排，但由于市场与价格机制的缺失，无法及时反映生产和消费的需求变化，导致生产效率低下、产出品质差，大量浪费社会资源。\n\n其次，集权主义导致决策不民主、无法有效响应社会变化。在苏联，政治权力高度集中，政府掌握了所有的经济、政治和文化资源，导致政府缺少对市场和民间经济的了解和把握，对自由市场竞争缺乏适应性，不能满足人民的需求和要求，使得国家和民间经济难以协同发展。\n\n此外，国有化也引起了一系列问题。在苏联，实行国有化，使得国家几乎垄断了所有工业、农业和服务业，使得政府需要大量的资源来维持，导致财政状况严峻，而由于国有化所造成的资源浪费和不必要的开支，也使得国家经济长期处于亏损状态。\n\n最后，苏联模式对人的自由与权利的限制过于严格。集权主义的体制使得个人自由被剥夺，特别是言论、新闻和出版自由受到了很大限制。这样的体制导致知识和信息的匮乏，影响了人们的思想和素质成长。这些问题最终导致苏联在1989年解体。\n\n苏联模式是指共产主义国家中的集权、计划经济、一党专政、个人权利受限等特征。为了防止苏联模式重现，以下是一些建议：\n\n1. 推动政治制度的改革，建立法治国家，保护人民的自由和权利，实现政治多元化。\n\n2. 建立市场经济，并实行合理有效的市场调节和竞争机制，以保持经济发展的稳定和可持续性。\n\n3. 加强教育，提高人民的

In [48]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages
)

In [49]:
response = completion.choices[0].message.content

In [50]:
response

'苏联模式：特点及其遗留问题\n\n苏联模式是一种集中式、计划经济体制的经济模式，主要原则是政府控制国有企业和资源，并通过中央计划来指导生产和分配。这种模式在苏联期间取得了一定的成功，但也面临了很多问题和挑战，最终在1991年苏联解体后终结。苏联模式的特点包括国有垄断、计划经济、等价交换、低效率、不合理的资源配置等。但随着时间的推移，这种模式逐渐暴露出许多问题。\n\n首先，计划经济效率低下。计划经济需要依靠庞大的机构和计算模型进行计划安排，但由于市场与价格机制的缺失，无法及时反映生产和消费的需求变化，导致生产效率低下、产出品质差，大量浪费社会资源。\n\n其次，集权主义导致决策不民主、无法有效响应社会变化。政治权力高度集中，导致政府缺少对市场和民间经济的了解和把握，对自由市场竞争缺乏适应性，不能满足人民的需求和要求，使得国家和民间经济难以协同发展。\n\n国有化也引起了一系列问题。国家几乎垄断了所有工业、农业和服务业，使得政府需要大量的资源来维持，导致财政状况严峻，同时由于国有化所造成的资源浪费和不必要的开支，国家经济长期处于亏损状态。\n\n苏联模式对人的自由与权利的限制过于严格。集权主义的体制剥夺了个人自由，使得个人的言论、新闻和出版受到很大限制。这样的体制导致知识和信息的匮乏，影响了人们的思想和素质成长。这些问题最终导致苏联在1989年解体。\n\n如何防止苏联模式重现？\n\n首先，推动政治制度的改革，建立法治国家，保护人民的自由和权利，实现政治多元化。其次，建立市场经济，并实行合理有效的市场调节和竞争机制，以保持经济发展的稳定和可持续性。加强教育，提高人民的素质和觉悟，使其能够理性思考问题并参与民主决策。加强跨国合作和交流，促进国际关系的和平化和多元化。同时，高度重视科技革命和现代化建设，提高国家的竞争力。\n\n如何解决苏联模式的遗留问题？\n\n逐步开放市场经济，采取“市场化改革”措施，使资源的配置更加有效率。同时，建立透明、公正、高效的政府管理机制，并加强政府与民间社会组织的合作与沟通。加强基础教育、职业培训和科技创新，赋予人们更多的知识和技能，提高其竞争力。同时，采取严厉的法律手段打击腐败现象，建立公正、高效的司法系统，维护人民的合法权益。\n\n与苏联模式相比，中国制定的社会主义建设方针注重农业优先发展，注重调动群众积极性，强调珠联璧合的地方发

In [40]:
prompt = "继续"

In [41]:
messages = []
message = {"role": "user", "content":  prompt}
messages.append(message)

In [42]:
messages

[{'role': 'user', 'content': '继续'}]

In [43]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages
)

In [44]:
response = completion.choices[0].message.content

In [45]:
response

'我不知道你指的是什么，能否提供更多的背景和信息让我更好地理解和回答你的问题呢？'